In [141]:
import pandas as pd
import numpy as np

In [142]:
df = pd.read_excel("C:\Data_Engineer_Study\멀캠_PJT_2\태블로 데이터\전체데이터관리(1).xlsx", sheet_name = "개발 채용 예측")

In [143]:
df

,년/월,시도,산업_대분류,구인인원,구직건수,취업건수,사업체 수,실업급여 수급자 수,각 지역별 인구,GDP(실질),금리,CLI 지수,소비자 심리지수
0,201801,서울,"농업, 임업 및 어업",0,0,0,254,0,9851767,1812.0,1.50,100.37300,110.7
1,201801,서울,광업,0,0,0,25,0,9851767,1812.0,1.50,100.37300,110.7
2,201801,서울,제조업,33,0,1,32866,0,9851767,1812.0,1.50,100.37300,110.7
3,201801,서울,"전기, 가스, 증기 및 공기조절 공급업",1,0,0,96,0,9851767,1812.0,1.50,100.37300,110.7
4,201801,서울,"수도, 하수 및 폐기물 처리, 원료 재생업",0,0,0,396,0,9851767,1812.0,1.50,100.37300,110.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23455,202212,제주,"협회 및 단체, 수리 및 기타 개인 서비스업",0,0,0,1441,0,678159,1968.8,3.25,98.75089,90.2
23456,202212,제주,가구 내 고용활동 및 달리 분류되지 않은 자가소비 생산활동,0,0,0,0,0,678159,1968.8,3.25,98.75089,90.2
23457,202212,제주,국제 및 외국기관,0,0,0,1,0,678159,1968.8,3.25,98.75089,90.2
23458,202212,제주,분류불능,0,0,16,0,0,678159,1968.8,3.25,98.75089,90.2


In [144]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_squared_log_error as msle


def get_scores_f(model, xtrain, xtest, ytrain, ytest):
    pred1 = model.predict(xtrain)   # train
    pred2 = model.predict(xtest)    # test

    # 음수가 있어도 상관없음
    A1 = r2_score(ytrain, pred1)
    A2 = r2_score(ytest, pred2)


    B = mae(ytest, pred2)
    C = mse(ytest, pred2)
    # msle, rmsle 는 음수가 있어서는 안됨
    pred2 = np.where(pred2<0, 0, pred2) # pred2값이 0보다 작은경우 0으로 대체, 아닌경우 그대로 pred2값..
    D = msle(ytest, pred2)
    E = np.sqrt(B)   # rmsle
    data = [round(x, 4) for x in [A1, A2, B, C, E]]
    names = 'r2_train r2_test mae mse rmsle'.split()
    scores = pd.Series(data, index=names)
    return  scores

def make_models_f(xtrain, xtest, ytrain, ytest, n=300):
    temp = pd.DataFrame()

    # 선형회귀 
    model1 = LinearRegression().fit(xtrain, ytrain)
    temp['model1'] = get_scores_f(model1, xtrain, xtest, ytrain, ytest)

    # 의사결정트리
    model2 = DecisionTreeRegressor(random_state=0).fit(xtrain, ytrain)  
    temp['model2'] = get_scores_f(model2, xtrain, xtest, ytrain, ytest)

    for d in range(3, 9):
        model2 = DecisionTreeRegressor(max_depth=d, random_state=0).fit(xtrain, ytrain)
        temp[f'model2_{d}'] = get_scores_f(model2, xtrain, xtest, ytrain, ytest)

    # 랜덤포레스트 
    model3 = RandomForestRegressor(n, random_state=0).fit(xtrain, ytrain)
    temp['model3'] = get_scores_f(model3, xtrain, xtest, ytrain, ytest)

    for d in range(3, 9):
        model3 = RandomForestRegressor(n, max_depth=d, random_state=0).fit(xtrain, ytrain)
        temp[f'model3_{d}'] = get_scores_f(model3, xtrain, xtest, ytrain, ytest)

    # XGB
    model4 = XGBRegressor(objective='reg:squarederror').fit(xtrain, ytrain)
    temp['model4'] = get_scores_f(model4, xtrain, xtest, ytrain, ytest)

    temp = temp.T
    temp.insert(2, 'diff', (temp['r2_train'] - temp['r2_test']).abs())

    return temp

In [145]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23460 entries, 0 to 23459
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   년/월         23460 non-null  int64  
 1   시도          23460 non-null  object 
 2   산업_대분류      23460 non-null  object 
 3   구인인원        23460 non-null  int64  
 4   구직건수        23460 non-null  int64  
 5   취업건수        23460 non-null  int64  
 6   사업체 수       23460 non-null  int64  
 7   실업급여 수급자 수  23460 non-null  int64  
 8   각 지역별 인구    23460 non-null  int64  
 9   GDP(실질)     23460 non-null  float64
 10  금리          23460 non-null  float64
 11  CLI 지수      23460 non-null  float64
 12  소비자 심리지수    23460 non-null  float64
dtypes: float64(4), int64(7), object(2)
memory usage: 2.3+ MB


In [146]:
# target 데이터인 '구인인원' 제외
# '년/월'의 경우 영향x -> 제외
df_X = df[['시도', '산업_대분류', '구직건수', '취업건수', '사업체 수', '실업급여 수급자 수',
       '각 지역별 인구', 'GDP(실질)', '금리', 'CLI 지수', '소비자 심리지수']].values

# 타겟 데이터
df_Y = df[['구인인원']].values

In [147]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [148]:
df_X

array([['서울', '농업, 임업 및 어업', 0, ..., 1.5, 100.373, 110.7],
       ['서울', '광업', 0, ..., 1.5, 100.373, 110.7],
       ['서울', '제조업', 0, ..., 1.5, 100.373, 110.7],
       ...,
       ['제주', '국제 및 외국기관', 0, ..., 3.25, 98.75089, 90.2],
       ['제주', '분류불능', 0, ..., 3.25, 98.75089, 90.2],
       ['제주', '해당없음', 55, ..., 3.25, 98.75089, 90.2]], dtype=object)

In [149]:
# '시도' -> 원핫인코딩 실시

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
df_X = np.array(ct.fit_transform(df_X))

In [150]:
# '산업_대분류' -> 원핫인코딩 실시

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [17])], remainder='passthrough')
df_X = np.array(ct.fit_transform(df_X))

In [151]:
df_X[0]

array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0, 0, 254, 0, 9851767, 1812.0, 1.5, 100.373, 110.7],
      dtype=object)

In [152]:
# 'GDP(실질)', '금리', 'CLI 지수', '소비자 심리지수'.... -> 정규화 실시(값 범위 0~1)
scaler = MinMaxScaler()
dfX_scale = scaler.fit_transform(df_X)
dfX_scale

array([[0.        , 0.        , 0.        , ..., 0.36363636, 0.4165242 ,
        1.        ],
       [0.        , 0.        , 0.        , ..., 0.36363636, 0.4165242 ,
        1.        ],
       [0.        , 0.        , 0.        , ..., 0.36363636, 0.4165242 ,
        1.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.04024227,
        0.45187166],
       [0.        , 0.        , 0.        , ..., 1.        , 0.04024227,
        0.45187166],
       [0.        , 0.        , 0.        , ..., 1.        , 0.04024227,
        0.45187166]])

- 훈련용 Xtrain(독립변수), Ytrain(종속변수) 분리

In [153]:
x_train, x_test, y_train, y_test = train_test_split(dfX_scale, df_Y,
                                                    test_size = 0.3,
                                                    random_state = 1)
print([x.shape for x in [x_train, x_test, y_train, y_test]])

[(16422, 49), (7038, 49), (16422, 1), (7038, 1)]


In [154]:
models = make_models_f(x_train, x_test, y_train, y_test)

c:\Python38\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Python38\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Python38\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Python38\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimato

In [155]:
print(models.sort_values('rmsle').head(10))

          r2_train  r2_test    diff     mae      mse   rmsle
model3      0.9910   0.9251  0.0659  1.5554  73.1153  1.2472
model4      0.9973   0.9387  0.0586  1.6041  59.9033  1.2665
model3_8    0.9782   0.9263  0.0519  1.6919  71.9905  1.3007
model2_8    0.9774   0.9110  0.0664  1.7918  86.8674  1.3386
model2      1.0000   0.9140  0.0860  1.8150  83.9849  1.3472
model3_7    0.9742   0.9227  0.0515  1.8289  75.4388  1.3524
model2_7    0.9742   0.9340  0.0402  1.8405  64.4003  1.3566
model3_6    0.9690   0.9203  0.0487  2.0476  77.8284  1.4310
model2_6    0.9662   0.9091  0.0571  2.1083  88.8119  1.4520
model3_5    0.9596   0.9123  0.0473  2.3651  85.5994  1.5379


- 일부 업종(농업..)의 경우 구인, 구직, 취업인원의 대부분이 0으로 되어있어 업종 제외 필요성..

In [156]:
job = pd.DataFrame()
for i in df['산업_대분류'].unique():
    job[i] = round(df[df['산업_대분류'] == f'{i}'][['구인인원', '구직건수', '취업건수']].mean(),2)

In [157]:
# 구인인원 기준 공공행정까지 컷해서 다시 머신러닝 돌리기
cut = job.T['구인인원'].sort_values(ascending = False)
cut

정보통신업                               58.00
제조업                                 16.85
도매 및 소매업                             7.77
전문, 과학 및 기술 서비스업                     6.36
사업시설 관리, 사업 지원 및 임대 서비스업             2.20
건설업                                  2.11
운수 및 창고업                             0.96
협회 및 단체, 수리 및 기타 개인 서비스업             0.63
보건업 및 사회복지 서비스업                      0.50
금융 및 보험업                             0.45
공공행정, 국방 및 사회보장 행정                   0.25
농업, 임업 및 어업                          0.24
부동산업                                 0.14
숙박 및 음식점업                            0.14
전기, 가스, 증기 및 공기조절 공급업                0.12
수도, 하수 및 폐기물 처리, 원료 재생업              0.05
국제 및 외국기관                            0.01
광업                                   0.00
교육서비스업                               0.00
예술, 스포츠 및 여가 관련 서비스업                 0.00
가구 내 고용활동 및 달리 분류되지 않은 자가소비 생산활동     0.00
분류불능                                 0.00
해당없음                                 0.00
Name: 구인인원, dtype: float64

In [158]:
cut.index

Index(['정보통신업', '제조업', '도매 및 소매업', '전문, 과학 및 기술 서비스업',
       '사업시설 관리, 사업 지원 및 임대 서비스업', '건설업', '운수 및 창고업',
       '협회 및 단체, 수리 및 기타 개인 서비스업', '보건업 및 사회복지 서비스업', '금융 및 보험업',
       '공공행정, 국방 및 사회보장 행정', '농업, 임업 및 어업', '부동산업', '숙박 및 음식점업',
       '전기, 가스, 증기 및 공기조절 공급업', '수도, 하수 및 폐기물 처리, 원료 재생업', '국제 및 외국기관', '광업',
       '교육서비스업', '예술, 스포츠 및 여가 관련 서비스업', '가구 내 고용활동 및 달리 분류되지 않은 자가소비 생산활동',
       '분류불능', '해당없음'],
      dtype='object')

In [159]:
cut_idx = ['정보통신업', '제조업', '도매 및 소매업', '전문, 과학 및 기술 서비스업',
       '사업시설 관리, 사업 지원 및 임대 서비스업', '건설업', '운수 및 창고업',
       '협회 및 단체, 수리 및 기타 개인 서비스업', '보건업 및 사회복지 서비스업', '금융 및 보험업',
       '공공행정, 국방 및 사회보장 행정']

---

# 1. cut_idx기준 머신러닝 실행

In [286]:
df_cut = df[df['산업_대분류'].isin(cut_idx)]
df_cut

,년/월,시도,산업_대분류,구인인원,구직건수,취업건수,사업체 수,실업급여 수급자 수,각 지역별 인구,GDP(실질),금리,CLI 지수,소비자 심리지수
2,201801,서울,제조업,33,0,1,32866,0,9851767,1812.0,1.50,100.37300,110.7
5,201801,서울,건설업,6,0,0,53694,0,9851767,1812.0,1.50,100.37300,110.7
6,201801,서울,도매 및 소매업,38,0,0,125584,0,9851767,1812.0,1.50,100.37300,110.7
7,201801,서울,운수 및 창고업,1,0,0,7956,0,9851767,1812.0,1.50,100.37300,110.7
9,201801,서울,정보통신업,303,0,9,26974,0,9851767,1812.0,1.50,100.37300,110.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23449,202212,제주,"전문, 과학 및 기술 서비스업",1,0,0,1561,3,678159,1968.8,3.25,98.75089,90.2
23450,202212,제주,"사업시설 관리, 사업 지원 및 임대 서비스업",0,0,0,1448,8,678159,1968.8,3.25,98.75089,90.2
23451,202212,제주,"공공행정, 국방 및 사회보장 행정",0,0,0,126,0,678159,1968.8,3.25,98.75089,90.2
23453,202212,제주,보건업 및 사회복지 서비스업,0,0,0,2066,0,678159,1968.8,3.25,98.75089,90.2


In [288]:
cut_X = df_cut[['시도', '산업_대분류', '취업건수', '사업체 수', '실업급여 수급자 수',
       '각 지역별 인구', 'GDP(실질)', '금리', 'CLI 지수', '소비자 심리지수']].values

cut_Y = df_cut[['구인인원']].values

In [289]:
# 시도 인코딩
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
cut_X = np.array(ct.fit_transform(cut_X))
cut_X

# 산업대분류 인코딩
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [17])], remainder='passthrough')
cut_X = np.array(ct.fit_transform(cut_X))
cut_X

array([[0.0, 0.0, 0.0, ..., 1.5, 100.373, 110.7],
       [1.0, 0.0, 0.0, ..., 1.5, 100.373, 110.7],
       [0.0, 0.0, 0.0, ..., 1.5, 100.373, 110.7],
       ...,
       [0.0, 1.0, 0.0, ..., 3.25, 98.75089, 90.2],
       [0.0, 0.0, 0.0, ..., 3.25, 98.75089, 90.2],
       [0.0, 0.0, 0.0, ..., 3.25, 98.75089, 90.2]], dtype=object)

In [290]:
# 'GDP(실질)', '금리', 'CLI 지수', '소비자 심리지수'.... -> 정규화 실시(값 범위 0~1)
scaler = MinMaxScaler()
cut_X_scale = scaler.fit_transform(cut_X)
cut_X_scale

array([[0.        , 0.        , 0.        , ..., 0.36363636, 0.4165242 ,
        1.        ],
       [1.        , 0.        , 0.        , ..., 0.36363636, 0.4165242 ,
        1.        ],
       [0.        , 0.        , 0.        , ..., 0.36363636, 0.4165242 ,
        1.        ],
       ...,
       [0.        , 1.        , 0.        , ..., 1.        , 0.04024227,
        0.45187166],
       [0.        , 0.        , 0.        , ..., 1.        , 0.04024227,
        0.45187166],
       [0.        , 0.        , 0.        , ..., 1.        , 0.04024227,
        0.45187166]])

In [291]:
len(cut_Y)

11220

In [292]:
x_train, x_test, y_train, y_test = train_test_split(cut_X_scale, cut_Y,
                                                    test_size = 0.3,
                                                    random_state = 1)
print([x.shape for x in [x_train, x_test, y_train, y_test]])

[(7854, 36), (3366, 36), (7854, 1), (3366, 1)]


In [293]:
models = make_models_f(x_train, x_test, y_train, y_test)

c:\Python38\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Python38\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Python38\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Python38\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimato

In [294]:
print(models.sort_values('rmsle').head(10))

          r2_train  r2_test    diff     mae       mse   rmsle
model3      0.9842   0.9222  0.0620  3.3147  209.9730  1.8206
model4      0.9974   0.9034  0.0940  3.5423  260.5799  1.8821
model3_8    0.9706   0.9216  0.0490  3.7571  211.4351  1.9383
model3_7    0.9660   0.9201  0.0459  3.9963  215.5725  1.9991
model2      1.0000   0.8658  0.1342  4.1771  362.1153  2.0438
model3_6    0.9590   0.9179  0.0411  4.3754  221.6488  2.0917
model2_8    0.9762   0.8707  0.1055  4.4992  348.7754  2.1211
model2_7    0.9701   0.8670  0.1031  4.7612  358.8764  2.1820
model3_5    0.9434   0.9160  0.0274  4.9286  226.7232  2.2200
model2_6    0.9552   0.8520  0.1032  5.5870  399.2611  2.3637


---

# 2. 가장 많은 정보통신업만 가지고 실행

In [160]:
df_it = df[df['산업_대분류'] == '정보통신업']
df_it

,년/월,시도,산업_대분류,구인인원,구직건수,취업건수,사업체 수,실업급여 수급자 수,각 지역별 인구,GDP(실질),금리,CLI 지수,소비자 심리지수
9,201801,서울,정보통신업,303,0,9,26974,0,9851767,1812.0,1.50,100.37300,110.7
32,201801,부산,정보통신업,22,0,1,1589,0,3467055,1812.0,1.50,100.37300,110.7
55,201801,대구,정보통신업,25,0,0,1168,0,2474309,1812.0,1.50,100.37300,110.7
78,201801,인천,정보통신업,6,0,1,1057,0,2950771,1812.0,1.50,100.37300,110.7
101,201801,광주,정보통신업,23,0,0,737,0,1464037,1812.0,1.50,100.37300,110.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23354,202212,전북,정보통신업,30,0,0,872,40,1769607,1968.8,3.25,98.75089,90.2
23377,202212,전남,정보통신업,31,0,0,714,18,1817697,1968.8,3.25,98.75089,90.2
23400,202212,경북,정보통신업,15,0,0,973,39,2600492,1968.8,3.25,98.75089,90.2
23423,202212,경남,정보통신업,22,0,2,1092,60,3280493,1968.8,3.25,98.75089,90.2


In [162]:
# target 데이터인 '구인인원' 제외
# '구직건수'의 경우 모든 값이 0 -> 제외 
# '년/월'의 경우 영향x -> 제외
# '정보통신업'만 있으므로 산업 대분류는 제외
X = df_it[['시도', '취업건수', '사업체 수', '실업급여 수급자 수',
       '각 지역별 인구', 'GDP(실질)', '금리', 'CLI 지수', '소비자 심리지수']]

# 타겟 데이터
Y = df_it[['구인인원']]

In [168]:
# test 데이터셋
# '시도' -> 원핫인코딩 실시

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

# 정규화 실시(값 범위 0~1)
scaler = MinMaxScaler()
X_scale = scaler.fit_transform(X)

X_scale

array([[0.        , 1.        , 0.        , ..., 0.8       , 0.4165242 ,
        1.        ],
       [0.        , 1.        , 0.        , ..., 0.8       , 0.4165242 ,
        1.        ],
       [0.        , 1.        , 0.        , ..., 0.8       , 0.4165242 ,
        1.        ],
       ...,
       [0.        , 1.        , 0.        , ..., 0.2       , 0.60954698,
        0.9171123 ],
       [0.        , 1.        , 0.        , ..., 0.2       , 0.60954698,
        0.9171123 ],
       [0.        , 1.        , 0.        , ..., 0.4       , 0.55816548,
        0.81818182]])

In [170]:
x_train, x_test, y_train, y_test = train_test_split(X_scale, Y,
                                                    test_size = 0.3,
                                                    random_state = 1)
print([x.shape for x in [x_train, x_test, y_train, y_test]])

[(560, 27), (240, 27), (560, 1), (240, 1)]


In [171]:
models = make_models_f(x_train, x_test, y_train, y_test)

c:\Python38\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Python38\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Python38\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Python38\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimato

In [172]:
print(models.sort_values('rmsle').head(10))

          r2_train  r2_test    diff      mae        mse   rmsle
model1      0.9212   0.9365  0.0153  14.5240  1111.5318  3.8110
model3_7    0.9803   0.9308  0.0495  15.7688  1211.3762  3.9710
model3_8    0.9813   0.9287  0.0526  15.7946  1247.6390  3.9742
model3      0.9833   0.9287  0.0546  15.8462  1247.3950  3.9807
model3_6    0.9785   0.9268  0.0517  16.0375  1280.5932  4.0047
model3_5    0.9748   0.9262  0.0486  16.3004  1291.3638  4.0374
model3_4    0.9634   0.9267  0.0367  16.4851  1282.4562  4.0602
model2_8    0.9956   0.8898  0.1058  17.0126  1928.3562  4.1246
model2_5    0.9816   0.8868  0.0948  17.1567  1982.0718  4.1421
model3_3    0.9476   0.9191  0.0285  17.2937  1416.4630  4.1586


In [173]:
# 2번쨰로 성능이 좋은 랜덤포레스트 확인
model3 = RandomForestRegressor(300, max_depth = 7, random_state=0).fit(x_train, y_train)
get_scores_f(model3, x_train, x_test, y_train, y_test)

c:\Python38\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


r2_train       0.9803
r2_test        0.9308
mae           15.7688
mse         1211.3762
rmsle          3.9710
dtype: float64

In [222]:
pred = model3.predict(x_train)
pred = pd.DataFrame(pred).reset_index(drop = True)
y_train = y_train.reset_index(drop = True)
# train 예측값 
train_pred = pd.concat([y_train, pred], axis = 1)
train_pred

,구인인원,0
0,9,13.265110
1,35,27.519625
2,193,256.465858
3,21,23.125496
4,31,24.545655
...,...,...
555,50,50.599990
556,47,31.706984
557,4,14.193715
558,17,17.151566


In [225]:
train_pred['diff'] = abs(train_pred['구인인원'] - train_pred[0])

In [227]:
train_pred['diff'].mean()

8.158240642013217

In [228]:
pred = model3.predict(x_test)
pred = pd.DataFrame(pred).reset_index(drop = True)
y_test = y_test.reset_index(drop = True)
# test 예측값 
test_pred = pd.concat([y_test, pred], axis = 1)

test_pred['diff'] = abs(test_pred['구인인원'] - test_pred[0])
test_pred['diff'].mean()

15.768848975342822